In [1]:
from functools import partial
import scipy.integrate

%matplotlib inline 
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from tqdm.notebook import tqdm

import sys
sys.path.append('..')
# << modify sys.path to import from the parent directory

from dl.DGM import *
from equations.pdes import *
from equations.solvers import *
from pulse.pulse import Pulse2D
from pulse.utils import *
from train.trainer import *
from graphics.animation import *

In [ ]:
B = np.array([35/384, 0, 500/1113, 125/192, -2187/6784, 11/84])

In [ ]:
fc1 = nn.Linear(4, 4)
fc2 = nn.Linear(4, 4)
fc3 = nn.Linear(4, 4)
fc4 = nn.Linear(4, 4)

X = torch.rand(5, 4)
fc = np.array([fc1, fc2, fc3, fc4])
sigma = lambda x: x


class GenerativeModel(nn.Module):
    def __init__(self, images, mask, sigma):
        super().__init__()
        self.images = np.array([*images, None])
        self.sigma = sigma

        self.N, self.M = mask.shape
        self.probs = torch.ones((self.N, self.M))
        self.probs[~mask] = 0

        self.probs /= self.probs.sum(1)[:, None]
        self.probs = nn.Parameter(self.probs)

    def refreeze(self):
        self.ix = torch.multinomial(self.probs, 1).view(-1)
        self.ll = self.probs[torch.arange(self.N), self.ix]
        self.ll = self.ll.prod().log()
        self.main = np.take(self.images, self.ix)

    def forward(self, X):
        Y = X.clone()
        for i, l in zip(self.ix, self.main):
            if i >= self.M-1: continue
            # ? upsample(X) ?
            Y = self.sigma(l(Y) + X)
        return Y


def gen_mask(N, M):
    mask = torch.zeros(N, M)
    ix_even = (torch.arange(N) % 2).bool()
    mask[ix_even] = 1
    mask[~ix_even, :M//2] = 1
    mask = torch.cat((mask, torch.ones(N)[:, None]), 1)
    return mask.bool()

# Relativistic thin-foil electrodynamics

Sergey Rykovanov

## 1. Theory

Primarily based on Bulanov et al, Phys. Plasmas 20, 123114 (2013)

We assume that a foil is delta-like. Ions are at the moment considered immobile and located at $x=0$. Laser pulse propagates from the left side $t-x$, everything is normalized in the usual relativistic laser-plasma way  ($e=1$, $m_e=1$, $c=1$, $\omega_L=1$). The model deals with the motion of the foil in the self-consistent way. Radiation from the foil itself is considered in the equations of motion. First, we start with the wave equation and its solutions in the case of the delta-foil.

$$\partial_{tt} \vec a - \partial_{xx} \vec a = -\alpha \delta(x-x_e(t)) \vec v\qquad (*)$$

where $x_e(t)$ is the position of 1D delta-electron, $\vec v$ - its velocity, $\alpha=n_e\, d$ is the areal density of the foil with $n_e$ - assumed density and $d$ assumed thickness (for comparison with PIC simulations).

Green function for the 1D wave equation is:

$$G(t,t',x,x')=\frac{1}{2}\Theta\left(t-t'-\left|x-x'\right| \right)\mathrm{,}$$
where $\Theta(x)$ is the Heaviside step function. Using this Green function it is easy to obtain radiation produced by the motion of the electron in 1D (Lienard-Wiechert potentials analogs):

1) observer is to the left of the foil (for transverse electric field components):

$$\vec E(t,x)=\alpha\frac{\vec{v}(t^*)}{1+v_x(t^*)}$$

It is convenient to write in components:

$$E_y(t,x)=-B_z(t,x)$$
$$E_z(t,x)=B_y(t,x)$$

2) observer is to the right of the foil

$$\vec E(t,x)=\alpha\frac{\vec{v}(t^*)}{1-v_x(t^*)}$$

$$E_y(t,x)=B_z(t,x)$$
$$E_z(t,x)=-B_y(t,x)$$

Remembering that there is a laser pulse coming from the left, and that force from self-radiation is equal to the average of fields on the both sides of the foil, the total field components can be written in the following way:

$$\epsilon\equiv\frac{\alpha}{2}$$
$$E_x=R(x)$$
$$E_y=E_{y,L}+\epsilon \frac{u_y}{\gamma}\gamma_x^2$$
$$E_z=E_{z,L}+\epsilon \frac{u_z}{\gamma}\gamma_x^2$$
$$B_x=0$$
$$B_y=-E_{z,L}-\epsilon \frac{u_x u_z}{\gamma^2}\gamma_x^2$$
$$B_z=E_{y,L}+\epsilon \frac{u_x u_y}{\gamma^2}\gamma_x^2$$

Here, $R(x)$ is the model for the restoring force of the ions, typically $R(x)=\epsilon\, \mathrm{sign}(x)$, $u$ - 4-momentum components, $\gamma_x=\frac{1}{\sqrt{1-v_x^2}}$. We can now construct the usual electromagnetic tensor and use relativistic mechanics to treat electron motion:

$$\frac{du^{\mu}}{d\tau}=-F^{\mu\nu}u_{\nu}\mathrm{,}$$
where $\tau$ is the proper time. We then get:

$$\frac{d\gamma}{d\tau}=-E_xu_x-E_{y,L}u_y-E_{z,L}u_z-\epsilon\frac{\gamma_x^2}{\gamma}\left(u_y^2+u_z^2 \right)$$
$$\frac{du_x}{d\tau}=-\gamma E_x-H_{z,L}u_y+H_{y,L}u_z-\epsilon\frac{u_x\gamma_x^2}{\gamma^2}\left(u_y^2+u_z^2 \right)$$
$$\frac{du_y}{d\tau}=-E_{y,L}\left(\gamma-u_x\right) - \epsilon u_y$$
$$\frac{du_z}{d\tau}=-E_{z,L}\left(\gamma-u_x\right) - \epsilon u_z$$


New variables:

$$\xi\equiv t-x$$
$$h\equiv\gamma-u_x$$
$$u_\perp\equiv u_y^2 + u_z^2$$

Then
$$\frac{d\xi}{d\tau}=h$$

$$u_x=\gamma-h=\frac{1+u_\perp^2-h^2}{2h}$$

After some manipulations we get:

$$\frac{dh}{d\xi}=E_x-\epsilon\frac{u_\perp^2}{1+u_\perp^2}$$
$$\frac{dx}{d\xi}=\frac{1+u_\perp^2-h^2}{2h^2}$$
$$\frac{du_y}{d\xi}=-E_{y,L}-\epsilon \frac{u_y}{h}$$
$$\frac{du_z}{d\xi}=-E_{z,L}-\epsilon \frac{u_z}{h}$$

Noticing, that $\frac{u_y}{h}=\frac{dy}{d\xi}$, we get the following set of master equations:

$$h'=E_x-\epsilon \frac{u_\perp^2}{1+u_\perp^2}$$
$$x'=\frac{1+u_\perp^2-h^2}{2h^2}$$
$$y'=\frac{u_y}{h}$$
$$z'=\frac{u_z}{h}$$
$$u_y=a_{y,L}-\epsilon y$$
$$u_z=a_{z,L}-\epsilon z$$

Below we solve these equations using RK4 method (`scipy.integrate.ode`)

Система дифференциальных ур-й
$$h'=\epsilon\left(\tanh\left(\frac{8x}{\theta}\right)-\frac{u_\perp^2}{1+u_\perp^2}\right)$$
$$x'=\frac{1+u_\perp^2-h^2}{2h^2}$$
$$y'=\frac{u_y}{h}$$
$$z'=\frac{u_z}{h}$$

Вспомогательные ур-я:
$$u_y=a_{y,L}-\epsilon y$$
$$u_z=a_{z,L}-\epsilon z$$
$$u^2_\perp = u_y^2 + u_z^2$$
Здесь $a_{y,L}$ и $a_{z,L}$ заданные цуги

## 2. Solving simple SODEs

$$\begin{align}
    \begin{pmatrix}
    \dot x\\ \dot y\\ \dot z
    \end{pmatrix}
    =
    \begin{pmatrix}
    x+2y+2z\\ 2x+y+2z\\ 2x+2y+z
    \end{pmatrix}
\end{align}$$

$$
\mathrm w(0) = \begin{pmatrix}2\\-1\\-1\end{pmatrix}
$$

In [ ]:
N = 100
net = RNNLikeDGM(1,3, as_array=False)
opt = optim.Adam(net.parameters(), 1e-3)

calc_loss = lambda t,x,y,z: torch.norm(D(x,t)-(x+2*y+2*z))
rbc = torch.tensor([2., -1., -1.])

history = []
for _ in trange(200):
    opt.zero_grad()
        
    t = torch.rand(N)
    t.requires_grad_(True)
    x,y,z = net(t).unbind(1)
    abc = net(t.new([0.]))
    
    loss = (1./N*(calc_loss(t,x,y,z)
         + calc_loss(t,y,x,z)
         + calc_loss(t,z,x,y))
         + torch.norm(rbc - abc))
    
    history.append(loss.item())
    loss.backward()
    
    opt.step()

In [ ]:
plt.figure()
plt.plot(history);
plt.yscale('log')

In [ ]:
plt.figure()
t = torch.linspace(0, 1)

x = net(t)[:,0].detach()
rx = 2*torch.exp(-t)

y = net(t)[:,1].detach()
ry = -torch.exp(-t)

plt.plot(t, x, label='x');
plt.plot(t, rx, '--', label='real x');

plt.plot(t, y, label='y');
plt.plot(t, ry, '--', label='real y');
plt.legend();

$$\left\{\begin{align}
&\dot x = x-y^2\\
&\dot y = y\\
&\dot z = x+y^2+z\\
\end{align}\right.$$

$$
\mathrm w(0) = \begin{pmatrix}0\\1\\0\end{pmatrix}
$$

In [ ]:
N = 100
net = RNNLikeDGM(1,3, growing=True, as_array=False)
opt = optim.Adam(net.parameters(), 1e-3)

rbc = torch.tensor([0., 1., 0.])
def calc_loss(t, x, y, z):
    out = (torch.norm(D(x,t) - (x-y**2))
           + torch.norm(D(y,t) - y)
           + torch.norm(D(z,t) - (x+y**2+z)))
    return out

history = []
for _ in trange(200):
    opt.zero_grad()
        
    t = torch.rand(N)
    t.requires_grad_(True)
    x,y,z = net(t).unbind(1)
    abc = net(t.new([0.]))
    
    loss = 1./N*calc_loss(t,x,y,z) + torch.norm(rbc - abc)
    history.append(loss.item())
    loss.backward()
    
    opt.step()

In [ ]:
plt.figure()
plt.plot(history);
plt.yscale('log')

In [ ]:
plt.figure()
t = torch.linspace(0, 1)

x = net(t)[:,0].detach()
rx = torch.exp(t) - torch.exp(2*t)

y = net(t)[:,1].detach()
ry = torch.exp(t)

z = net(t)[:,2].detach()
rz = t*torch.exp(t)

plt.plot(t, x, label='x');
plt.plot(t, rx, '--', label='real x');

plt.plot(t, y, label='y');
plt.plot(t, ry, '--', label='real y');

plt.plot(t, z, label='z');
plt.plot(t, rz, '--', label='real z');
plt.legend();

$$\left\{\begin{align}
&\dot x = -2x-y+37\sin t\\
&\dot y = -4x-5y\\
\end{align}\right.$$

$$
\mathrm w(0) = \begin{pmatrix}-5\\4\end{pmatrix}
$$

In [ ]:
N = 200
net = RNNLikeDGM(1,2, as_array=False)
opt = optim.Adam(net.parameters(), 1e-3)

rbc = torch.tensor([-5., 4.])
def calc_loss(t, x, y, z):
    out = (torch.norm(D(x,t) - (-2*x-y+37*torch.sin(t)))
           + torch.norm(D(y,t) - (-4*x-5*y)))
    return out

history = []
for _ in trange(6000):
    opt.zero_grad()
        
    t = 10*torch.rand(N)
    t.requires_grad_(True)
    x,y = net(t).unbind(1)
    abc = net(t.new([0.]))
    
    loss = 1./N*calc_loss(t,x,y,z) + torch.norm(rbc - abc)
    history.append(loss.item())
    loss.backward()
    
    opt.step()

In [ ]:
plt.figure()
plt.plot(history);

In [ ]:
plt.figure()
t = 10*torch.linspace(0, 1)

x = net(t)[:,0].detach()
rx = 10*torch.exp(-t) + 16*torch.sin(t) - 15*torch.cos(t)
plt.plot(t, x, label='x');
plt.plot(t, rx, '--', label='real x');

y = net(t)[:,1].detach()
ry = -10*torch.exp(-t) - 10*torch.sin(t) + 14*torch.cos(t)
plt.plot(t, y, label='y');
plt.plot(t, ry, '--', label='real y');

plt.legend();

---

## 3. Solving with a Neural Network

In [ ]:
from torch import optim
torch.manual_seed(123)
p_t = Pulse2D(pulse_torch, ((a_0y, a_0z), (xi_0y, xi_0z), tau))                    
w = torch.tensor([1./96, 1.], device=device)

xi_lims = (0, 20*math.pi)
pde = ThinFoilSODE(                                                                
        rbc, p_t, theta, xi_lims=xi_lims, device=device)            
net = RNNLikeDGM(1,4, as_array=False).to(device)                                   
opt = optim.SGD(net.parameters(), 8e-3, weight_decay=5e-7)                         
#sch = optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, 500, eta_min=5e-7)       
sch = optim.lr_scheduler.StepLR(opt, 1, gamma=.999)       
                                                                                   
trainer = SepLossTrainer(net, pde, opt, sch)

In [ ]:
xi = pde.sampleBatch(20)
R_do, R_bc = pde.computeResiduals(xi, net)

In [ ]:
trainer.exponentialWarmUp(w, 256, 1000, (1e-6, 1), range_test=True, explosion_ratio=2.)

In [2]:
# --> Problem Setting
N = 500

a_0y = 20.
a_0z = 20.
xi_0y = 0.
xi_0z = .5*math.pi

tau = 10*math.pi
theta = .02*math.pi
eps = 200*theta

device = 'cuda' if torch.cuda.is_available() else 'cpu'

rbc = torch.tensor([0., 0., 0., 1.])
# <--

In [ ]:
!scp -r zhores:SR_research/savings ../..

In [ ]:
hfn = '../../runs/history1.npz'
wfn = '../../runs/weights1.pt'
content = np.load(hfn)
history, histories = content.values()

net = RNNLikeDGM(1, 4, as_array=False)
net.load_state_dict(torch.load(wfn, map_location='cpu'))

In [ ]:
print(history[-1])
plt.plot(histories[0][100:]);

In [ ]:
xi = torch.linspace(0, 20*math.pi, 10000)
xyzh = net(xi)

In [ ]:
plt.plot(xi, xyzh[:, 1].detach());

$a_{y,L}$ and $a_{y,L}$ are the solution projections of the homogeneous form of the equation $(*)$, that can be set as follows (like Sergey proposed it last time)

$$
    a_{y,L} = a_{0y}\exp\left(-\frac{\xi^2}{2\tau}\right)\sin(\xi + \xi_{0y})\\
    a_{z,L} = a_{0z}\exp\left(-\frac{\xi^2}{2\tau}\right)\sin(\xi + \xi_{0z})
$$

For numerical solution obtained with RK4, he uses $a$ of the following form:

$$
    a_{0y}\sin^2\frac{\pi\xi}{\tau}\;\sin(\xi-\frac{\tau}{2}+\xi_{0y})\\
    a_{0z}\sin^2\frac{\pi\xi}{\tau}\;\sin(\xi-\frac{\tau}{2}+\xi_{0z})
$$

for $\xi$ values: $0 \le \xi < \tau$ (for others it is 0)

$$
    \partial_\xi a = 2\frac{a_0\pi}{\tau}\sin\frac{\pi\xi}{\tau}\cos\frac{\pi\xi}{\tau}
    \sin(\xi - \frac{\tau}{2} + \xi_0) + a_0\sin^2\frac{\pi\xi}{\tau}\cos(\xi-\frac{\tau}{2} + \xi_0)\\
    \simeq a_0\sin^2\frac{\pi\xi}{\tau}\cos(\xi-\frac{\tau}{2} + \xi_0)
$$

$$
    E^2 \simeq a_{0y}^2\sin^4\frac{\pi\xi}{\tau}\cos^2(\xi-\frac{\tau}{2} + \xi_0y)
    + a_{0z}^2\sin^4\frac{\pi\xi}{\tau}\cos^2(\xi-\frac{\tau}{2} + \xi_0z)
$$

In [ ]:
y = a_0y*np.sin(np.pi*x/tau)**2*np.sin(x - tau/2 + xi_0y)/dx
dy1 = a_0y*np.sin(np.pi*x/tau)**2*np.cos(x - tau/2 + xi_0y)
dy2 = (a_0y*np.sin(np.pi*x/tau)**2*np.cos(x - tau/2 + xi_0y)
       + 2*a_0y*np.pi/tau*np.sin(np.pi*x/tau)*np.cos(np.pi*x/tau)*np.sin(x - tau/2 + xi_0y))

In [ ]:
da_y = a_0y*np.sin(np.pi*x/tau)**2*np.cos(x - tau/2 + xi_0y)
da_z = a_0z*np.sin(np.pi*x/tau)**2*np.cos(x - tau/2 + xi_0z)

In [ ]:
plt.plot(da_y**2 + da_z**2)

In [ ]:
plt.plot(np.gradient(y))
plt.plot(dy1)
plt.plot(dy2);

## 4. Numerical Solution

In [3]:
### ---> Problem Setting
dxi = 1e-5
xi_lims = (0., 20*np.pi)
n_steps = np.ceil(xi_lims[1]/dxi)+1
xi = np.linspace(*xi_lims, int(n_steps))

a_0y = 20.
a_0z = 20.
xi_0y = 0.
xi_0z = .5*np.pi
tau = 10*np.pi  # pulse duration

theta = .02*np.pi  # thickness
eps = 200*theta

rbc = torch.tensor([0., 0., 0., 1.])
### <--- Problem Setting

In [4]:
p1 = Pulse2D(pulse_scalar, ([a_0y, a_0z], [xi_0y, xi_0z], tau))
p2 = Pulse2D(pulse_scalar, ([a_0y, a_0z], [xi_0y-np.pi, xi_0z-np.pi], tau, 3))

In [ ]:
p1 = Pulse2D(pulse_np, ([a_0y, a_0z], [xi_0y, xi_0z], tau))
p2 = Pulse2D(pulse_np, ([a_0y, a_0z], [xi_0y-np.pi, xi_0z-np.pi], tau, 3))

In [ ]:
plt.figure()
y1 = pulse_np(torch.linspace(0, 20*np.pi, 1000), a_0y, xi_0y, tau, 1)
plt.plot(y1)
y2 = pulse_np(torch.linspace(0, 20*np.pi, 1000), a_0y, xi_0y, tau, 2)
plt.plot(y2);

In [ ]:
plt.figure()
y1 = pulse(torch.linspace(0, 20*np.pi, 1000), a_0y, xi_0y, tau, 1)
y2 = pulse(torch.linspace(0, 20*np.pi, 1000), a_0y, xi_0y-np.pi, tau, 3)
plt.plot(y1+y2);

# 4 par-s: ampl1, ampl2, dphi, ellipt
# 2 cases:
# - circular
# - linear
# E = const (conservation)

In [ ]:
xi = np.linspace(0, 20*math.pi, 1000)

In [5]:
solver = ThinFoilSolver(rbc, p1, eps, theta, pulse2=None, scheme='new', xi_lims=xi_lims)
x,y,z,h = solver(xi)

In [ ]:
hfn = '../../runs/history1.npz'
wfn = '../../runs/weights1.pt'
content = np.load(hfn)
history, histories = content.values()

net = RNNLikeDGM(1, 4, as_array=False)
net.load_state_dict(torch.load(wfn, map_location='cpu'))
x,y,z,h = net(torch.Tensor(xi)).T.detach().numpy()

In [ ]:
dx_dxi = np.gradient(x) / dxi
print(r'max derivative of x:', f'{max(dx_dxi):.2f}')

fig, ax1 = plt.subplots(figsize=(8, 4))
ax1.plot(xi, x, c='blue')
ax1.set_ylabel(r'$x$', c='blue', size=16)
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_xlabel(r'$\xi$')

ax2 = ax1.twinx()
ax2.plot(xi, dx_dxi, c='orange', alpha=.4);
ax2.set_ylabel(r'$\frac{dx}{d\xi}$', c='orange', size=16)
ax2.tick_params(axis='y', labelcolor='orange')

plt.tight_layout();

In [ ]:
mask = (xi > 36.31) & (xi < 36.32)
plt.plot(xi[mask], x[mask]);

In [6]:
u_x = np.gradient(x, xi) * h
u_y = np.gradient(y, xi) * h
u_z = np.gradient(z, xi) * h

gamma = np.sqrt(1 + u_x**2 + u_y**2 + u_z**2)
v_x = u_x / gamma
gamma_x = 1. / np.sqrt(1 - v_x**2)

In [ ]:
sw = 1.
fig, ax1 = plt.subplots(figsize=(8,4))
xaxis = xi/2/np.pi + sw*x/2/np.pi
ax1.plot(
    xaxis, x/2/np.pi, 'k',
    lw=2, label=r'$\frac{x^\prime}{\lambda_L}$')
ax1.set_xlabel(r'$t/T_L$', size=16)
ax1.set_ylabel(r'$\frac{x^\prime}{\lambda_L}$', size=26)
ax1.set_xlim(0,10)

ax2 = ax1.twinx()
ax2.plot(xaxis, gamma_x, 'g', label=r'$\gamma_x$')
ax2.plot(xaxis, u_y, 'b', label=r'$u_y$')
ax2.plot(xaxis, u_z, 'r', label=r'$u_z$')
ax2.set_ylabel(r'$u_y,\;u_z,\;\gamma_x$', size=16)

lines = ax1.lines + ax2.lines
lbls = map(lambda x: x.get_label(), lines)
plt.legend(lines, lbls, loc=0)
plt.tight_layout();

## Cells copied from the updated version

In [ ]:
t = xi + x
t_detector = t + x
t_uni = np.linspace(xi[0]+x[0], xi[-1]+x[-1], len(xi))


E_y_detector = eps * u_y / gamma / (1+v_x)
E_z_detector = eps * u_z / gamma / (1+v_x)

E_y_interp = np.interp(t, t_detector, E_y_detector)
E_z_interp = np.interp(t, t_detector, E_z_detector)

plt.plot(t_detector/2/np.pi, E_y_detector)
plt.title("Foil's electic field")
plt.ylabel('E')
plt.xlabel('t');

In [ ]:
plt.plot(t)
plt.plot(t_detector)

In [ ]:
spectrum_y = np.fft.fft(E_y_interp)
spectrum_z = np.fft.fft(E_z_interp)

w = np.fft.fftfreq(len(E_y_interp), d=t[1]-t[0])

plt.semilogy(w*2*np.pi, abs(spectrum_y)/abs(spectrum_y).max())
plt.semilogy(w*2*np.pi, abs(spectrum_z)/abs(spectrum_z).max())

plt.title("Spectrum of the foil's electric field")
plt.ylim(1e-2,1)
plt.xlim(0,10);

In [ ]:
filter_center = 10.0
filter_width = 3.

bandgap_filter_minus = np.exp(
    -(w*2*np.pi - filter_center)**16 / filter_width**16)
bandgap_filter_plus = np.exp(
    -(w*2*np.pi + filter_center)**16 / filter_width**16)

bandgap_filter = bandgap_filter_minus + bandgap_filter_plus
plt.semilogy(w*2*np.pi, abs(spectrum_y)/abs(spectrum_y).max())
plt.semilogy(
    w*2*np.pi,
    abs(bandgap_filter*spectrum_y) / abs(spectrum_y).max())
plt.semilogy(w*2*np.pi, bandgap_filter)

sp_y_filt = bandgap_filter*spectrum_y
sp_z_filt = bandgap_filter*spectrum_z
plt.xlim(-30,30)
plt.ylim(1e-3,1);

In [ ]:
plt.plot(w*2*np.pi, abs(sp_y_filt))
plt.plot(w*2*np.pi, abs(sp_z_filt), alpha=.7)
plt.xlim(0,30);

In [ ]:
y_filt = np.real(np.fft.ifft(sp_y_filt))
z_filt = np.real(np.fft.ifft(sp_z_filt))

plt.plot(y_filt, z_filt)
plt.xlim(-15,15);
plt.ylim(-15,15);

In [ ]:
plt.plot(t/2/np.pi, abs(y_filt)**2 + abs(z_filt)**2);
plt.xlim(5.0,5.3)

## Rectified cells 

In [ ]:
from grid_search import spectrum_integral

In [ ]:
t = xi + 2*x
t_uni = np.linspace(xi[0]+x[0], xi[-1]+x[-1], len(xi))

E_y = eps * u_y / gamma / (1+v_x)
E_z = eps * u_z / gamma / (1+v_x)

E_yi = np.interp(t, t_uni, E_y)
E_zi = np.interp(t, t_uni, E_z)

plt.plot(t_uni/2/np.pi, E_yi)
plt.title("Foil's electric field")
plt.ylabel('E')
plt.xlabel('t');

In [ ]:
spec_y = np.fft.fft(E_yi)
spec_z = np.fft.fft(E_zi)

dt = t_uni[1] - t_uni[0]
w = np.fft.fftfreq(len(t_uni), d=dt)

plt.semilogy(w*2*np.pi, abs(spec_y)/abs(spec_y).max())
plt.semilogy(w*2*np.pi, abs(spec_z)/abs(spec_z).max())

plt.title("Spectrum of the foil's electric field")
plt.ylim(1e-2,1)
plt.xlim(0,10);

In [ ]:
fc = 10. # filter's center
bw = 3.  # filter's width

Fl = np.exp(-(w*2*np.pi - fc)**16 / bw**16)
Fr = np.exp(-(w*2*np.pi + fc)**16 / bw**16)
F = Fl + Fr

spf_y = F*spec_y
spf_z = F*spec_z

plt.semilogy(w.real*2*np.pi, abs(spec_y)/abs(spec_y).max())
plt.semilogy(w.real*2*np.pi, abs(F*spec_y) / abs(spec_y).max())
plt.semilogy(w.real*2*np.pi, F)

plt.xlim(-30,30)
plt.ylim(1e-3,1);

In [ ]:
i_upper = math.ceil(dt*len(xi)*(fc+bw)/(2*np.pi))
i_lower = math.floor(dt*len(xi)*(fc-bw)/(2*np.pi))

intf_spy = abs(spf_y[i_lower:i_upper]).sum()
intf_spz = abs(spf_z[i_lower:i_upper]).sum()

Q = (intf_spy**2 + intf_spz**2)**.5

In [ ]:
plt.plot(w*2*np.pi, 3.5*abs(spf_y))
plt.plot(w*2*np.pi, 3.5*abs(spf_z), alpha=.7)
plt.xlim(0,30);
#plt.savefig('../../figures/solspec.png')

In [ ]:
y_f = np.real(np.fft.ifft(spf_y))
z_f = np.real(np.fft.ifft(spf_z))

plt.plot(y_f, z_f);